# P19 Benchmark

### Import modules

In [1]:
import os
from datetime import datetime
from pathlib import Path

import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter

os.chdir('..')
from raindrop.raindrop import Raindrop

### Load data

In [49]:
PROCESSED_PATH = Path('./p19/data/processed_data')
static_feat_names = np.load(PROCESSED_PATH / 'labels_demogr.npy')
ts_feat_names = np.load(PROCESSED_PATH / 'labels_ts.npy')
inputs = np.load(PROCESSED_PATH / 'PT_dict_list_6.npy', allow_pickle=True)
labels = np.load(PROCESSED_PATH / 'arr_outcomes_6.npy').squeeze()

ts_inputs = np.array([inp['arr'] for inp in inputs])[:, :, :, np.newaxis]
static_inputs = np.array([inp['extended_static'] for inp in inputs])
times = np.array([inp['time'] for inp in inputs]).squeeze()
lengths = np.array([inp['length'] for inp in inputs])

ts_inputs = torch.tensor(ts_inputs, dtype=torch.float32)
static_inputs = torch.tensor(static_inputs, dtype=torch.float32)
times = torch.tensor(times, dtype=torch.float32)
lengths = torch.tensor(lengths)
labels = torch.tensor(labels, dtype=torch.int64)

### Split data

In [50]:
np.random.seed(42)

num_samples = ts_inputs.shape[0]
idxs = np.arange(num_samples)
np.random.shuffle(idxs)

train_idxs, val_idxs, test_idxs = idxs[:(s1 := int(num_samples*0.8))], idxs[s1: (s2 := int(num_samples*0.9))], idxs[s2:]

train_ts_inp, val_ts_inp, test_ts_inp = ts_inputs[train_idxs], ts_inputs[val_idxs], ts_inputs[test_idxs]
train_static_inp, val_static_inp, test_static_inp = static_inputs[train_idxs], static_inputs[val_idxs], static_inputs[test_idxs]
train_times, val_times, test_times = times[train_idxs], times[val_idxs], times[test_idxs]
train_lengths, val_lengths, test_lengths = lengths[train_idxs], lengths[val_idxs], lengths[test_idxs]
train_lbls, val_lbls, test_lbls = labels[train_idxs], labels[val_idxs], labels[test_idxs]

In [51]:
print("P19 Summary")
print(f"Num Samples: {num_samples}\n\tTrain: {train_idxs.shape[0]}\n\tVal: {val_idxs.shape[0]}\n\tTest: {test_idxs.shape[0]}")

pos = [(int(t := labels.sum()), 100 * t / labels.shape[0])] + \
      [(t := int(lbls.sum()), 100 * t / lbls.shape[0]) for lbls in (train_lbls, val_lbls, test_lbls)]
print(f"Classes: {pos[0][0]} {pos[0][1]:.2f}%")
for lbl, (t, p) in zip(['Train', 'Val', 'Test'], pos):
    print(f"\t{lbl}: {t} {p:.2f}%")

P19 Summary
Num Samples: 38803
	Train: 31042
	Val: 3880
	Test: 3881
Classes: 1626 4.19%
	Train: 1626 4.19%
	Val: 1299 4.18%
	Test: 175 4.51%


In [52]:
from torch.utils.data import Dataset, DataLoader
from torch import Tensor

class P19Dataset(Dataset):
    def __init__(self,
                 ts_inp: Tensor,
                 times: Tensor,
                 lengths: Tensor,
                 static_inp: Tensor,
                 labels: Tensor):
        self.ts_inp = ts_inp
        self.times = times
        self.lengths = lengths
        self.static_inp = static_inp
        self.labels = labels

    def __len__(self) -> int:
        return len(self.labels)
    
    def __getitem__(self, idx: int | slice):
        ts_inp = self.ts_inp[idx]

        # Create mask
        mask = torch.zeros(ts_inp.shape[:-1], dtype=bool)
        if isinstance(idx, int):
            mask[:self.lengths[idx]] = 1
        else:
            for idx, length in enumerate(self.lengths[idx]):
                mask[idx, :length] = 1

        return ts_inp, self.times[idx], mask, self.static_inp[idx], self.labels[idx]

In [53]:
train_ds = P19Dataset(train_ts_inp, train_times, train_lengths, train_static_inp, train_lbls)
val_ds = P19Dataset(val_ts_inp, val_times, val_lengths, val_static_inp, val_lbls)
test_ds = P19Dataset(test_ts_inp, test_times, test_lengths, test_static_inp, test_lbls)

train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=64, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=64, shuffle=True)


### Define the model

In [54]:
raindrop = Raindrop(num_sensors=34,
                 obs_dim=1,
                 obs_embed_dim=4,
                 pe_emb_dim=16,
                 timesteps=60,
                 out_dim=128,
                 num_layers=2,
                 inter_sensor_attn_dim=16,
                 temporal_attn_dim=16,
                 prune_rate=0.5)



class RaindropClassifier(nn.Module):
    def __init__(self,
                 rd_model: Raindrop,
                 static_dim: int,
                 static_proj_dim: int,
                 cls_hidden_dim: int,
                 classes: int):
        super().__init__()
        self.static_dim = static_dim
        self.static_proj_dim = static_proj_dim
        self.cls_hidden_dim = cls_hidden_dim
        self.classes = classes

        self.rd_model = rd_model
        self.static_proj = nn.Linear(static_dim, static_proj_dim)

        rd_out_dim = self.rd_model.out_dim * self.rd_model.num_sensors
        self.cls = nn.Sequential(
            nn.Linear(rd_out_dim + static_proj_dim, cls_hidden_dim),
            nn.Linear(cls_hidden_dim, classes))


    def forward(self, x_ts, times, mask, x_static):
        ts_emb, reg_loss = self.rd_model(x_ts, times, mask)
        ts_emb = ts_emb.view(ts_emb.shape[0], -1)

        static_emb = self.static_proj(x_static)

        emb = torch.concat([ts_emb, static_emb], dim=-1) 
        return F.softmax(self.cls(emb), dim=-1), reg_loss
    
rd_cls = RaindropClassifier(raindrop,
                            static_dim=6,
                            static_proj_dim=34,
                            cls_hidden_dim=128,
                            classes=2)


### Training utilities

In [57]:

class RaindropLoss(nn.Module):
    def __init__(self, reg_weight: float):
        super().__init__()
        
        self.ce_loss = nn.CrossEntropyLoss()
        self.reg_weight = reg_weight

    def forward(self, predictions, targets, reg_loss):
        ce_loss = self.ce_loss(predictions, targets)
        reg_loss *= self.reg_weight
        return ce_loss + reg_loss
    
loss_fn = RaindropLoss(reg_weight=0.02)

In [58]:
NUM_EPOCHS = 20
LOSS_TRAIN_LOG_FREQ = 100


optim = torch.optim.Adam(rd_cls.parameters(), lr=0.001)

### Train loop

In [59]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    for i, data in enumerate(train_dl):
        ts_inp, times, mask, static_inp, labels = data

        optim.zero_grad()
        outputs, reg_loss = rd_cls.forward(ts_inp, times, mask, static_inp)

        loss = loss_fn(outputs, labels, reg_loss)
        loss.backward()

        optim.step()

        running_loss += loss.item()
        if i % LOSS_TRAIN_LOG_FREQ == LOSS_TRAIN_LOG_FREQ-1:
            last_loss = running_loss / LOSS_TRAIN_LOG_FREQ
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_dl) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [60]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/p19/p19_trainer_{}'.format(timestamp))
epoch_number = 0

best_vloss = 1e6

for epoch in range(NUM_EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    rd_cls.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)


    running_vloss = 0.0
    rd_cls.eval()

    with torch.no_grad():
        for i, vdata in enumerate(val_dl):
            vinputs, vlabels = vdata
            voutputs = rd_cls(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(rd_cls.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:
CALC_INTER_SENSOR_ATTENTION
h: torch.Size([64, 4, 60, 34, 4])
pe: torch.Size([64, 60, 16])
lay_idx: 0
CALC_INTER_SENSOR_ATTENTION
h: torch.Size([64, 4, 60, 34, 4])
pe: torch.Size([64, 60, 16])
lay_idx: 1
CALC_INTER_SENSOR_ATTENTION
h: torch.Size([64, 4, 60, 34, 4])
pe: torch.Size([64, 60, 16])
lay_idx: 0
CALC_INTER_SENSOR_ATTENTION
h: torch.Size([64, 4, 60, 34, 4])
pe: torch.Size([64, 60, 16])
lay_idx: 1


KeyboardInterrupt: 